# Verifying our quantized model

So, we have our model ready. But how do we need that it will indeed work as intended?

In [1]:
import numpy as np
from tensorflow.lite.python.interpreter import Interpreter
from tensorflow.examples.tutorials.mnist import input_data

Setup the TFLite interpreter:

In [2]:
model_path = './data/mnist_cnn_quant.tflite'
with open(model_path, 'rb') as f:
    model_content = f.read()

interpreter = Interpreter(model_content=bytes(model_content))

In [3]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

input_index = input_details[0]['index']
output_index = output_details[0]['index']

input_scale, input_zero_point = input_details[0]['quantization']
output_scale, output_zero_point = output_details[0]['quantization']
input_stats = (input_index, input_scale, input_zero_point)
output_stats = (output_index, output_scale, output_zero_point)

# Do not forget this!
interpreter.allocate_tensors()


[{'name': 'input', 'index': 14, 'shape': array([  1, 784], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.007874015718698502, 128)}]
[{'name': 'prob', 'index': 17, 'shape': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.00390625, 0)}]


Define the feed-forward process for the interpreter:

In [4]:
def do_inference(data, interpreter, input_stats, output_stats): 
    input_index, input_scale, input_zero_point = input_stats
    output_index, output_scale, output_zero_point = output_stats

    test_input = np.array(data / input_scale + input_zero_point, dtype=np.uint8).reshape(1, -1)
    interpreter.set_tensor(input_index, test_input)
    interpreter.invoke()
 
    output_data = interpreter.get_tensor(output_index)[0]
    output_data = (np.array(output_data, dtype=np.float32) - output_zero_point)* output_scale
    return output_data 

Load the data:

In [5]:
mnist = input_data.read_data_sets('MNIST-data', one_hot=True)
test_data, test_labels = mnist.test.images, mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Invoke the interpreter for all the samples:

In [6]:
predictions = [do_inference(x, interpreter, input_stats, output_stats) for x in test_data]

In [7]:
predictions = np.asarray(predictions).argmax(1)

In [8]:
true_labels = test_labels.argmax(1)
accuracy = np.sum(true_labels==predictions)/len(true_labels)

In [9]:
print("Accuracy :", 100*accuracy, "%")

Accuracy : 99.28 %
